#Group SOMETHING 1D ML project
1. Loy Pek Yong 1004475


#Part 1

In [4]:
# import libraries
import pandas as pd
import numpy as np

# read data
ES_train = pd.read_csv("./ES/train", sep=" ", header=None, names=["word", "label"], encoding="utf-8", quoting=3)
ES_dev_in = pd.read_csv("./ES/dev.in", sep=" ", header=None, names=["word"], encoding="utf-8", quoting=3)
ES_dev_out = pd.read_csv("./ES/dev.out", sep=" ", header=None, names=["word", "label"], encoding="utf-8", quoting=3)

RU_train = pd.read_csv("./RU/train", sep=" ", header=None, names=["word", "label"], encoding="utf-8", quoting=3)
RU_dev_in = pd.read_csv("./RU/dev.in", sep=" ", header=None, names=["word"], encoding="utf-8", quoting=3)
RU_dev_out = pd.read_csv("./RU/dev.out", sep=" ", header=None, names=["word", "label"], encoding="utf-8", quoting=3)



In [7]:
ES_train

,word,label
0,Estuvimos,O
1,hace,O
2,poco,O
3,mi,O
4,pareja,O
...,...,...
31171,junto,O
31172,con,O
31173,la,O
31174,carta,O


In [15]:

    print(word, value)

Estuvimos O
hace O
poco O
mi O
pareja O
y O
yo O
comiendo O
y O
resultó O
todo O
muy O
bien O
, O
tanto O
la O
comida B-positive
, O
el O
vino B-positive
, O
el O
trato B-positive
, O
la O
decoración B-positive
… O
nos O
gustó O
todo O
mucho O
. O
Por O
poner O
algún O
pero O
, O
quizá O
el O
jamón B-negative
no O
era O
todo O
lo O
" O
ibérico O
" O
que O
cabía O
esperar O
. O
Bien O
lo O
sabe O
el O
autor O
del O
blog O
. O
) O
Comida B-positive
exquisita O
. O
Restaurante B-positive
diferente O
, O
creativo O
y O
agradable O
. O
Si O
no O
has O
probado O
sus O
carnes B-positive
te O
estas O
perdiendo O
algo O
muy O
grande O
! O
En O
resumen O
, O
comida B-positive
bien-muy O
bien O
, O
servicio B-positive
correcto O
y O
profesional O
02-12- O
2012 O
elegimos O
este O
restaurante B-negative
por O
los O
comentarios O
, O
pero O
ha O
sido O
una O
Grandisima O
Decepción O
. O
Salimos O
encantadas O
del O
restaurante B-positive
. O
Ubicación O
Comimos O
muy O
bien O
y O
bien O
atendidos O

KeyboardInterrupt: 

In [19]:
# Part 1 Point 1
# function that returns the emission parameters with input of training set

def train_estimate_emission_parameters(train):
    # word_column = train["word"]
    # label_column = train["label"]

    label_count = {}
    emission_count = {}

    for row in range(train.shape[0]):
        word = ES_train.iloc[row, 0]
        label = ES_train.iloc[row, 1]
        if label not in label_count:
            label_count[label] = 1
        else:
            label_count[label] += 1

        if (word, label) not in emission_count:
            emission_count[(word, label)] = 1
        else:
            emission_count[(word, label)] += 1

    emission_parameter = {}
    for word, label in emission_count:
        emission_parameter[(word, label)] = emission_count[(word, label)] / label_count[label]

    return emission_parameter, emission_count, label_count

# debug
# emission_parameter, emission_count, label_count = train_estimate_emission_parameters(ES_train)
# for i in emission_parameter:
#     print(i, emission_parameter[i])

In [5]:


# Part 1 Point 2
# function that returns the emission parameters with input set
def upgraded_estimate_emission_parameters(input, emission_parameter, emission_counts, label_counts):
    # predict the label of the test set
    k = 1
    special_word = '#UNK#'    
    #train word list is all the first argument in keys of emission counts
    train_word_list= [word for word, label in emission_counts.keys()]
    for i in range(len(input["word"])):
        word = input["word"][i]
        if word not in train_word_list:
            input["word"][i] = special_word
            # k += 1

    for word in input["word"]:
        for label in label_counts:
            if word == special_word:
                emission_parameter[(word, label)] = k / (label_counts[label] + k)
            else:

                if (word, label) in emission_parameter:
                    emission_parameter[(word, label)] = emission_counts[(word, label)] / label_counts[label]

    return emission_parameter

# Part 1 Point 3

def sentiment_analysis(training_set, test_set):
    emission_parameter, emission_count, label_count = train_estimate_emission_parameters(training_set)
    emission_parameter = upgraded_estimate_emission_parameters(test_set, emission_parameter, emission_count, label_count)
    test_set["label"] = ""

    #label equals to argmax of emission parameter
    for row in range(len(test_set)):
        word = test_set.iloc[row]["word"]
        max = 0
        max_label = ""
        for label in label_count:
            if (word, label) in emission_parameter:
                if emission_parameter[(word, label)] > max:
                    max = emission_parameter[(word, label)]
                    max_label = label
        test_set.loc[row, "label"] = max_label
                    
    
    return test_set


def precision(predicted, test_set):
    # calculate the precision
    correct = 0
    total = 0
    for row in range(len(test_set)):
        if test_set.iloc[row]["label"] == predicted.iloc[row]["label"]:
            correct += 1
        total += 1
    return correct/total if total > 0 else 0

def recall(predicted, compare_set):
    # calculate the recall
    correct = 0
    total = 0
    for row in range(len(compare_set)):
        if compare_set.iloc[row]["label"] == predicted.iloc[row]["label"]:
            correct += 1
        total += 1
    return correct/total if total > 0 else 0

def f1_score(precision, recall):
    # calculate the f1 score
    return 2*precision*recall/(precision+recall)

ES_dev_p1_out = sentiment_analysis(ES_train, ES_dev_in)
ES_dev_p1_out.to_csv("./ES/dev.p1.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)

RU_dev_p1_out = sentiment_analysis(RU_train, RU_dev_in)
RU_dev_p1_out.to_csv("./RU/dev.p1.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)



In [6]:
print("ES:")
p = precision(ES_dev_p1_out, ES_dev_out)
r = recall(ES_dev_p1_out, ES_dev_out)
f1 = f1_score(p, r)
print("precision is:", p)
print("recall is:",  r)
print("f1 score is:", f1)

print("RU:")
p = precision(RU_dev_p1_out, RU_dev_out)
r = recall(RU_dev_p1_out, RU_dev_out)
f1 = f1_score(p, r)
print("precision is:", p)
print("recall is:",  r)
print("f1 score is:", f1)


ES:
precision is: 0.6307977736549165
recall is: 0.6307977736549165
f1 score is: 0.6307977736549165
RU:
precision is: 0.6425861283958112
recall is: 0.6425861283958112
f1 score is: 0.6425861283958112
